In [5]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [6]:

# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())


In [7]:

# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

array([0, 1, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0,
       1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1], dtype=int64)

In [9]:
pred_onx

array([0, 1, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0,
       1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1], dtype=int64)

In [8]:
y_test

array([0, 1, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 2, 2, 1, 1, 0, 1, 1, 0, 0,
       1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1])

In [12]:
# Accuracy 평가지표
from sklearn.metrics import accuracy_score

# Predict에 대한 Accuracy Score
print('accuracy_score', float(accuracy_score(y_test, pred_onx))) 

accuracy_score 0.9736842105263158


In [2]:
import onnxmltools
from keras.layers import Input, Dense, Add
from keras.models import Model

# N: batch size, C: sub-model input dimension, D: final model's input dimension
N, C, D = 2, 3, 3

# Define a sub-model, it will become a part of our final model
sub_input1 = Input(shape=(C,))
sub_mapped1 = Dense(D)(sub_input1)
sub_model1 = Model(inputs=sub_input1, outputs=sub_mapped1)

# Define another sub-model, it will become a part of our final model
sub_input2 = Input(shape=(C,))
sub_mapped2 = Dense(D)(sub_input2)
sub_model2 = Model(inputs=sub_input2, outputs=sub_mapped2)

# Define a model built upon the previous two sub-models
input1 = Input(shape=(D,))
input2 = Input(shape=(D,))
mapped1_2 = sub_model1(input1)
mapped2_2 = sub_model2(input2)
sub_sum = Add()([mapped1_2, mapped2_2])
keras_model = Model(inputs=[input1, input2], output=sub_sum)

# Convert it! The target_opset parameter is optional.
onnx_model = onnxmltools.convert_keras(keras_model, target_opset=7) 


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


c:\users\urse\anaconda3\envs\untitled\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ad...)`


In [3]:
onnx_model

ir_version: 3
producer_name: "keras2onnx"
producer_version: "1.6.1"
domain: "onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "input_4"
    input: "dense_2/kernel:0"
    output: "transformed_tensor1"
    name: "dense_2"
    op_type: "MatMul"
    doc_string: ""
    domain: ""
  }
  node {
    input: "transformed_tensor1"
    input: "dense_2/bias:0"
    output: "biased_tensor_name1"
    name: "Add2"
    op_type: "Add"
    domain: ""
  }
  node {
    input: "input_3"
    input: "dense_1/kernel:0"
    output: "transformed_tensor"
    name: "dense_1"
    op_type: "MatMul"
    doc_string: ""
    domain: ""
  }
  node {
    input: "transformed_tensor"
    input: "dense_1/bias:0"
    output: "biased_tensor_name"
    name: "Add1"
    op_type: "Add"
    domain: ""
  }
  node {
    input: "biased_tensor_name"
    input: "biased_tensor_name1"
    output: "add_1/add:01"
    name: "Add"
    op_type: "Add"
    doc_string: ""
    domain: ""
  }
  node {
    input: "add_1/add:01"
    o

In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(X_train, y_train)

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("logreg_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    
import onnxruntime as rt
sess = rt.InferenceSession("logreg_iris.onnx")
input_name = sess.get_inputs()[0].name

pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]